In [1]:
import numpy as np
from collections import defaultdict
from collections import Counter

In [2]:
# SET THIS TO LOAD EXAMPLE DATA INSTEAD
# example = True
example = False

if example:
    fname = 'example.txt'
else:
    fname = 'input.txt'

with open(fname) as f:
    lines = f.readlines()

connections = [[thing.strip() for thing in line.split('-')] for line in lines]

In [3]:
uniques = set([item for sublist in connections for item in sublist])
is_small = dict([(name, name[0].islower()) for name in uniques])
is_small

{'WP': False,
 'ak': True,
 'lg': True,
 'qx': True,
 'iw': True,
 'js': True,
 'start': True,
 're': True,
 'JG': False,
 'bj': True,
 'end': True,
 'CG': False}

In [4]:
neighbors = defaultdict(list)
for a, b in connections:
    neighbors[a].append(b)
    neighbors[b].append(a)

neighbors

defaultdict(list,
            {'re': ['js', 'ak', 'bj', 'qx', 'lg', 'WP'],
             'js': ['re', 'start', 'bj', 'CG'],
             'qx': ['CG', 'ak', 're', 'start'],
             'CG': ['qx', 'ak', 'js', 'lg', 'bj'],
             'start': ['js', 'bj', 'qx'],
             'bj': ['start', 'js', 're', 'CG', 'WP'],
             'ak': ['qx', 're', 'CG', 'lg', 'WP', 'end'],
             'lg': ['ak', 'CG', 're', 'JG', 'end', 'iw'],
             'WP': ['ak', 'end', 're', 'bj'],
             'end': ['WP', 'ak', 'lg'],
             'JG': ['lg'],
             'iw': ['lg']})

In [5]:
%%time

def get_paths(current_node, temp_already_visited, depth=0):
    already_visited = set(temp_already_visited)
    
    already_visited.add(current_node)
    
    if current_node == 'end':
        return ['end']
    
    # visit all children
#     print(f"at {current_node}, depth {depth}, to visit {neighbors[current_node]}")
    paths = []
    for neighbor in neighbors[current_node]:
        
        if (is_small[neighbor] and neighbor not in already_visited) or not is_small[neighbor] and neighbor != "start" :
#             print(f"from {current_node} (depth {depth}) visiting {neighbor}")
            temp_new_set = set(already_visited)
            paths += get_paths(neighbor, temp_new_set, depth+1)

#     for neighbor in neighbors[current_node]:
#         if is_small[neighbor]: 
#             already_visited.add(neighbor)
        
    return [(current_node+','+path) for path in paths]

len(get_paths('start', set(['start'])))

Wall time: 31 ms


3230

In [6]:
%%time

def get_paths(current_node, temp_already_visited, one_double_so_far=False, depth=0):
    already_visited = set(temp_already_visited)
    
    if current_node in already_visited:
        one_double_so_far = True
        visit_same_small = False
        
    # only keep track of small
    if is_small[current_node]:
        already_visited.add(current_node)
    
    if current_node == 'end':
        return ['end']
    
    # visit all children
#     print(f"at {current_node}, depth {depth}, to visit {neighbors[current_node]}")

    
    
    paths = []
    for neighbor in neighbors[current_node]:
        
        if (is_small[neighbor] and ((neighbor not in already_visited) or (neighbor in already_visited and visit_same_small)) or not is_small[neighbor]) and neighbor != "start" :
#             print(f"from {current_node} (depth {depth}) visiting {neighbor}")
            temp_new_set = set(already_visited)
            paths += get_paths(neighbor, temp_new_set, one_double_so_far=True, depth=depth+1)

#     for neighbor in neighbors[current_node]:
#         if is_small[neighbor]: 
#             already_visited.add(neighbor)
        
    return [(current_node+','+path) for path in paths]

len(get_paths('start', set(['start'])))

UnboundLocalError: local variable 'visit_same_small' referenced before assignment

In [7]:
%%time

def get_paths(current_node, temp_visited_count, depth=0):
    
    visited_count = defaultdict(int, temp_visited_count)
    if is_small[current_node]: 
        visited_count[current_node] += 1 # don't count non smalls
    
    if current_node == 'end':
        return ['end']
    
    paths = []
    
    revisit_smalls = Counter(visited_count.values())[2] < 1 # no more than one '2' in visited
    already_visited = visited_count.keys()
    
    for neighbor in neighbors[current_node]:
        
        if ((is_small[neighbor] and neighbor not in already_visited) or \
             (is_small[neighbor] and neighbor in already_visited and revisit_smalls ) or neighbor == 'end' or not is_small[neighbor]) and neighbor != "start":
            paths += get_paths(neighbor, visited_count.copy(), depth+1)
     
    return [(current_node+','+path) for path in paths]

len(set(get_paths('start', {'start':0})))

Wall time: 2.44 s


83475

In [8]:
# for efficiency now

In [9]:
%%time

def get_paths(current_node, already_visited=set(), revisit_small_allowed=True, depth=0):
    
    
    already_visited = already_visited.copy()
    
    if is_small[current_node]: 
        already_visited.add(current_node)
    
    if current_node == 'end':
        return ['end']
    
    paths = []
        
    for neighbor in neighbors[current_node]:
        if neighbor == "start":
            continue
        if neighbor in already_visited:
            if revisit_small_allowed:
                paths += get_paths(neighbor, already_visited, revisit_small_allowed=False, depth=depth+1)
            else:
                continue
        else: 
            paths += get_paths(neighbor, already_visited, revisit_small_allowed, depth=depth+1)
             
    return [(current_node+','+path) for path in paths]

len(set(get_paths('start')))

Wall time: 697 ms


83475